## Situation and Task

We need to build a model that comes up with price/yield difference between two peer bonds, various business products depends on this ability.

- **Background**: Two motivations
    - To enable real-time pricing: once bond A traded, though the price adjustment we can immediately update bond B's price.
    - For muni, come up with the $4\%$ and $3\%$ AAA curves from the published $5\%$ AAA curve. 

## Action

We structured it as a machine-learning project: built a machine learning model where given the attributes of the two bonds, fit/predicts the price/yield difference between bond B and bond A.

- The **attributes**, or $X$, include, time-to-maturity, first-call dates, coupons, ratings etc. of the two bonds, and the yield/price of bond A.
- $y$ **depends on the domain**: if it is muni, use YTW diff; corp IG, G spread diff; corp HY, price diff.
- **Data are in pairs**, and there is a somewhat involved procedure to define the pairs; see this [notebook](updating-illiquid-bond-px.ipynb).
- A **separate model is trained on each issuer**, in principle, unless there is not enough data: see the pairing procedure above.
- We also tried other machine learning models, such as random forests, regressions, etc., but MARS is chosen for these two reasons
    - It maintains **non-linearity** (which is almost inevitable) while having **better interpretability**: 
        - 'The model adds 15bps to the YTW difference since these two bonds are 5 years in maturity apart; yes I see you have an example where the difference is only 5bps, but I also have three other examples where the difference is 20bps, so 15bps by the model is not crazy'.
    - MARS does **simple yet automatic feature engineering** in a sense; see this [notebook](../../machine-learning/supervised-learning/MARS.ipynb) for more details.

## Resolution

Reduce pricing erros and make viable various new product lines.

- **Take-away**: Experience in MLOps that can carry over to other machine learning models.
    - The importance of **ML pipeline**: CI/CD, versioning of dataset to replay, packaging on different platform, etc.
    - The importance of **subject matter expert**: to determine the expectation of accuracy; to comment on examples and suggest new features (use AMT as an example); to help confirm on guardrails; why we need to care more about a particular slice of the data.
    - The proven way to conduct **ML Research**: how to tune hyperparameters to deal with under/overfitting; the importance of looking at the data before choosing the right ML model; the importance of error analysis, etc.

## Misc.: previous messy description

Bloomberg has the muni callable curve for 5% AAA issuers. It is a benchmark curve that investors look to to gauge the overall level of muni market yields. But while 5% was the prevailing rate in the muni market in the past, as the general rates plunged in recent years, investors are in strong demand of 4% or 3% muni callable curves.

So our task is, based on the 5% curve that we already have, to come up with the necessary adjustments to arrive at the 4% and 3% curves.

Naturally, to come up with the data, we looked where everyone would look - the first data we want to inspect are the pairs of muni bonds that are of different coupons but same call date and maturity date, and traded together.

Note that we had to form the pairs of the same issuer, to cancel out the idiosyncratic credit risk of the specific issuer. The reason is, even if two issuers are of the same AAA rating, one may consistently trade at a significant positive spread against another.

But to our dismay, if we impose that restriction, we quickly run into lack-of-data problems. As you may know, muni is an extremely illiquid market, since most investors are buy-and-hold. We find that day-on-day, we are lucky to find around 100 such pairs, and the yield differences on those pairs are rather noisy. We keep plowing on, and quickly decide that we can enlarge the data set by including non-AAA rating issuers (but add rating as a feature of the model). Also form pairs that differ in maturity and call date by a wider margin, say 5-10 years. In this way, we have a stabler data set each day. And more importantly, among the various machine learning models we try, they all have the capacity to learn wider data. In other words, the approach we take instead is to learn a broader model that can learn many types of pairs while making predictions only on specific pairs.

This then produces the further problem that optimizing the topline accuracy metric does not guarantee the precision for the slice of data that we care about, such as pairs of bonds that differ only in coupon, or new-issue pairs. For that, we tweaked the weighting in the cost function and assign more weights for pairs with similar call date, maturity and different coupon so that they have bigger say in the trained model.

At the end of the day, to accommodate interpretability, we choose the MARS model, which has the appeal like linear regression in that its trained coefficients have clear meanings and that MARS can learn non-linear relationships.

All in all, It is a typical machine learning application on a fixed income problem. We are glad that it has the potential to bring in hundreds of millions of extra revenues (estimated from the case of the 5% AAA curve).